## Using HGDL and the dask dashboard

In [1]:
!pip install ../

Processing /home/elliott/Documents/hgdl
  Created wheel for hgdl: filename=hgdl-1.0-py3-none-any.whl size=10741 sha256=72728472a4f6dde00f49aa702c58fb787f965acaf2df5ea12d6824cd4d725456
  Stored in directory: /tmp/pip-ephem-wheel-cache-o6mnqadt/wheels/6b/c9/23/792ad4bdbe555082c55c8811620c083ede102ae1e0a722bea4
Successfully built hgdl
  Attempting uninstall: hgdl
    Found existing installation: hgdl 1.0
    Uninstalling hgdl-1.0:
      Successfully uninstalled hgdl-1.0


In [2]:
from hgdl.hgdl import HGDL

In [3]:
HGDL?

Init signature: HGDL(*args, **kwargs)
Docstring:     
HGDL
    * Hybrid - uses both local and global optimization
    * G - uses global optimizer
    * D - uses deflation
    * L - uses local extremum localMethod
    Mandatory Parameters:
        * func - should return a scalar given a numpy array x
        -- note: use functools.partial if you have optional params
        * grad - gradient vector at x
        * bounds - numpy array of bounds in same format as scipy.optimize
    Optional Parameters:
        * Overall Parameters -----------------------------------
        * hess (None) - hessian array at x - you may need this depending on the local method
        * client - (None->HGDL initializes if None) dask.distributed.Client object
            -- this lets you interface with clusters via dask with Client(myCluster)
        * num_epochs (10) - the number of epochs. 1 epoch is 1 global step + 1 local run
        * numWorkers (logical cpu cores -1) - how many processes to use
        

Let's use the most simple version here on the rosenbrock function from scipy.optimize

In [4]:
from scipy.optimize import rosen, rosen_der, rosen_hess
import numpy as np

In [5]:
bounds = np.array([[-2,2.],[-4,45],[-3,3]])

In [6]:
from dask.distributed import Client

In [7]:
from dask.distributed import LocalCluster

In [8]:
c = Client(LocalCluster(n_workers=3, threads_per_worker=2, dashboard_address=':0'))

In [9]:
c.dashboard_link

'http://127.0.0.1:46299/status'

In [10]:
ls

01_basic_usage.ipynb              11_bump_function.ipynb  dask-worker-space/
02_understanding_hgdl_args.ipynb  12_global.ipynb         __init__.py*
03_basic_dashboard-Copy1.ipynb    13_newton.ipynb         scheduler.json


In [11]:
c.scheduler_file

In [12]:
%%time
hgdl = HGDL(rosen, rosen_der, bounds, hess=rosen_hess, client=c, num_individuals=10**3)

hgdl.get_best()

hgdl.get_final()

CPU times: user 60 s, sys: 7.04 s, total: 1min 7s
Wall time: 4min 6s


{'best_x': array([1.00000001, 1.00000002, 1.00000004]),
 'best_y': 1.190304529892752e-15,
 'minima_x': array([[1.00000001, 1.00000002, 1.00000004]]),
 'minima_y': array([1.19030453e-15]),
 'global_x': array([[ 0.07836477,  0.07505431,  0.07143454],
        [-0.58408731,  0.44585692,  0.13685647],
        [ 0.23114597, -0.13769978, -0.02365913],
        [-0.35327022,  0.01211651,  0.19297759],
        [ 0.92874526,  0.65289053,  0.75063829]]),
 'global_y': array([ 2.61282231,  4.29615025,  5.7201472 ,  7.79537846, 15.04375141])}

In [13]:
hgdl = HGDL(rosen, rosen_der, bounds, hess=rosen_hess, client=c, num_individuals=10**2)

Now I can open up the localhost on a new tab and see what's happening

In [14]:
hgdl.get_best()

{'best_x': array([1.00000001, 1.00000002, 1.00000004]),
 'best_y': 1.190304529892752e-15}

In [15]:
hgdl.get_final()

{'best_x': array([1.00000001, 1.00000002, 1.00000004]),
 'best_y': 1.190304529892752e-15,
 'minima_x': array([[1.00000001, 1.00000002, 1.00000004]]),
 'minima_y': array([1.19030453e-15]),
 'global_x': array([[ 0.51945058,  0.09069819,  0.16583997],
        [-0.68960821,  0.31122714, -0.14499328],
        [-0.33554902, -0.32454441, -0.40405736],
        [ 1.07458997,  1.27930635,  2.49607107],
        [-0.31253475,  0.98055592,  0.46652794]]),
 'global_y': array([  6.7507499 ,  11.87910942,  48.59448673,  75.49996289,
        104.16920923])}

### The true minimum is [1,1,1], so this is it, that's all that's strictly necessary.